## Mi Experiencia Adaptando CFR para Nocca-Nocca

### El problema que tuve

Nocca-Nocca tiene un espacio de estados gigantesco que hizo que CFR estándar se trabara por horas y se quedara sin memoria.

**Los problemas específicos que encontré**:
- Espacio de estados: millones de estados únicos (tablero 8x5, pilas de 3 niveles, 8 direcciones)
- CFR estándar crea un nodo por cada estado que visita → explosión de memoria
- Los juegos pueden ser muy largos (más de 100 movimientos)
- Factor de ramificación muy alto

### Lo que implementé para solucionarlo

Basándome en lo que leí en papers, implementé:

1. **Abstracción de estados**: Reducir el espacio de estados agrupando estados similares
2. **Monte Carlo CFR (MCCFR)**: Muestrear trayectorias en lugar de explorar todo
3. **CFR+ con clipping**: Truncar regrets negativos para convergencia más rápida
4. **Límites de memoria**: Prevenir explosión de nodos
5. **Evaluación heurística**: Para posiciones que no son terminales

In [ ]:
import numpy as np
import time
import os
from games.tictactoe.tictactoe import TicTacToe
from games.nocca_nocca.nocca_nocca import NoccaNocca
from games.kuhn.kuhn import KuhnPoker
from games.kuhn.kuhn_3player import KuhnPoker3Player
from games.leduc.leduc import LeducPoker
from agents.counterfactualregret_t import CounterFactualRegret
from agents.agent_random import RandomAgent

In [ ]:
def test_cfr_con_timeout(game, nombre_juego, iteraciones_entrenamiento=10, episodios_prueba=5, timeout_segundos=20, save_agents=True):
    """Prueba CFR con protección de timeout para evitar cuelgues"""
    
    try:
        agents = {}
        
        if 'NoccaNocca' in nombre_juego:
            max_depth = 5
        elif 'Kuhn' in nombre_juego or 'Leduc' in nombre_juego:
            max_depth = 25
        else:
            max_depth = 15
        
        for agent_id in game.agents:
            agents[agent_id] = CounterFactualRegret(game=game, agent=agent_id, max_depth=max_depth)
        
        inicio_tiempo = time.time()
        print(f"Entrenando CFR para {nombre_juego} ({iteraciones_entrenamiento} iteraciones, timeout: {timeout_segundos}s)...")
        
        for agent in agents.values():
            agent.train(niter=iteraciones_entrenamiento, timeout_seconds=timeout_segundos)
        
        tiempo_entrenamiento = time.time() - inicio_tiempo
        nodos_aprendidos = len(agents[game.agents[0]].node_dict)
        
        if nodos_aprendidos > 10000:
            print(f"MUCHOS: {nodos_aprendidos} nodos creados - posible explosión de estados")

        if save_agents:
            os.makedirs('trained_cfr_agents', exist_ok=True)
            for agent_id, agent in agents.items():
                filepath = f'trained_cfr_agents/{nombre_juego}_{agent_id}.pkl'
                agent.save_agent(filepath)
        
        juegos_exitosos = 0
        
        for episodio in range(episodios_prueba):
            try:
                game.reset()
                pasos = 0
                max_pasos = 50 if 'NoccaNocca' in nombre_juego else 200
                
                while not game.game_over() and pasos < max_pasos:
                    action = agents[game.agent_selection].action()
                    game.step(action)
                    pasos += 1
                
                if game.game_over():
                    juegos_exitosos += 1
                elif pasos >= max_pasos: 
                    print(f"Step limit: episode {episodio}, steps: {max_pasos}")

            except Exception as e:
                print(f"Episode error: {episodio}, {e}")
                continue
        
        tasa_exito = juegos_exitosos / episodios_prueba
        
        return True, agents, {
            'tiempo_entrenamiento': tiempo_entrenamiento,
            'nodos_aprendidos': nodos_aprendidos,
            'tasa_exito': tasa_exito,
            'juegos_exitosos': juegos_exitosos,
            'episodios_totales': episodios_prueba
        }
    except Exception as e:
        print(f"ERROR: {nombre_juego}, {e}")
        return False, None, {'error': str(e)}

def evaluar_cfr_vs_aleatorio(game, agentes_cfr, episodios=20):
    """Evalúa agentes CFR entrenados contra agentes aleatorios"""
    
    victorias_cfr = 0
    empates = 0
    
    for episodio in range(episodios):
        game.reset()
        
        if episodio % 2 == 0:
            agents = {
                game.agents[0]: agentes_cfr[game.agents[0]],
                game.agents[1]: RandomAgent(game=game, agent=game.agents[1])
            }
            if len(game.agents) > 2:
                agents[game.agents[2]] = RandomAgent(game=game, agent=game.agents[2])
            agente_cfr = game.agents[0]
        else:
            agents = {
                game.agents[0]: RandomAgent(game=game, agent=game.agents[0]),
                game.agents[1]: agentes_cfr[game.agents[1]]
            }
            if len(game.agents) > 2:
                agents[game.agents[2]] = RandomAgent(game=game, agent=game.agents[2])
            agente_cfr = game.agents[1]
        
        pasos = 0
        max_pasos = 50 if 'NoccaNocca' in str(type(game).__name__) else 200
        while not game.game_over() and pasos < max_pasos:
            try:
                action = agents[game.agent_selection].action()
                game.step(action)
                pasos += 1
            except:
                available = game.available_actions()
                if available:
                    action = np.random.choice(available)
                    game.step(action)
                    pasos += 1
        
        if game.game_over():
            rewards = {agent: game.reward(agent) for agent in game.agents}
            reward_cfr = rewards[agente_cfr]
            otras_rewards = [rewards[agent] for agent in game.agents if agent != agente_cfr]
            
            if reward_cfr > max(otras_rewards):
                victorias_cfr += 1
            elif reward_cfr == max(otras_rewards):
                empates += 1
    
    tasa_victoria = victorias_cfr / episodios
    
    return {
        'victorias_cfr': victorias_cfr,
        'empates': empates,
        'derrotas': episodios - victorias_cfr - empates,
        'tasa_victoria': tasa_victoria,
        'episodios_totales': episodios
    }

def cargar_y_evaluar_agente_guardado(game, nombre_juego, episodios=20):
    """Carga agentes CFR entrenados desde archivo y los evalúa"""
    
    agents_guardados = {}
    directorio = 'trained_cfr_agents'
    
    if not os.path.exists(directorio):
        print(f"Directorio {directorio} no encontrado. No hay agentes guardados.")
        return None
        
    for agent_id in game.agents:
        filepath = f'{directorio}/{nombre_juego}_{agent_id}.pkl'
        if os.path.exists(filepath):
            agente_cargado = CounterFactualRegret.load_trained_agent(filepath, game, agent_id)
            agents_guardados[agent_id] = agente_cargado
        else:
            print(f"Archivo {filepath} no encontrado para agente {agent_id}")
            return None
            
    if len(agents_guardados) != len(game.agents):
        print(f"No se pudieron cargar todos los agentes para {nombre_juego}")
        return None
        
    print(f"Agentes CFR cargados exitosamente para {nombre_juego}")
    
    resultados = evaluar_cfr_vs_aleatorio(game, agents_guardados, episodios)
    
    return {
        'agentes_cargados': agents_guardados,
        'evaluacion': resultados,
        'nodos_totales': sum(len(agent.node_dict) for agent in agents_guardados.values())
    }

In [ ]:
from agents.counterfactualregret_t import Node

class NoccaCFRAgent(CounterFactualRegret):
    """
    CFR adaptado para Nocca-Nocca
    
    Adaptaciones:
    1. Abstracción agresiva de estados
    2. Heurísticas de terminación temprana
    3. Límites de memoria
    4. Función de evaluación simplificada
    """
    
    def __init__(self, game, agent, max_nodes=800):
        super().__init__(game, agent, max_depth=6)  
        self.max_nodes = max_nodes
        self.abstract_states = {}
        
    def abstract_game_state(self, game):
        """
        Reduce el espacio de estados de Nocca-Nocca
        
        En lugar de rastrear posiciones exactas, rastrea:
        1. Conteos de piezas
        2. Distancia al objetivo (agrupada)  
        3. Si alguna pieza está bloqueada
        4. Fase del juego
        """
        if game.board is None:
            return "initial"
            
        board = game.board.squares
        
        black_pieces = []
        white_pieces = []
        
        for x in range(8):
            for y in range(5):
                for z in range(3):
                    if board[x, y, z] == 0:
                        black_pieces.append((x, y, z))
                    elif board[x, y, z] == 1:
                        white_pieces.append((x, y, z))
        
        black_count = len(black_pieces)
        white_count = len(white_pieces)
        
        if black_pieces:
            black_min_dist = min([7 - x for x, y, z in black_pieces])
            black_dist_bucket = min(black_min_dist // 2, 3)
        else:
            black_dist_bucket = 4
            
        if white_pieces:
            white_min_dist = min([x for x, y, z in white_pieces]) 
            white_dist_bucket = min(white_min_dist // 2, 3)
        else:
            white_dist_bucket = 4
        
        black_free = any(z == 2 or board[x, y, z+1] == -1 for x, y, z in black_pieces)
        white_free = any(z == 2 or board[x, y, z+1] == -1 for x, y, z in white_pieces)
        
        total_pieces = black_count + white_count
        if total_pieces >= 8:
            phase = "early"
        elif total_pieces >= 5:  
            phase = "mid"
        else:
            phase = "late"
        
        abstract_state = f"{phase}_B{black_count}W{white_count}_BD{black_dist_bucket}WD{white_dist_bucket}_BF{int(black_free)}WF{int(white_free)}"
        
        return abstract_state
    
    def observe(self, game):
        return self.abstract_game_state(game)
    
    def cfr_rec(self, game, agent, probability, depth=0):
        """Override recursión CFR con límites para Nocca-Nocca"""
        
        if depth >= self.max_depth:
            return self.evaluate_position(game, agent)
            
        if game.game_over():
            return self._safe_get_reward(game, agent)
        
        if len(self.node_dict) >= self.max_nodes:
            return self.evaluate_position(game, agent)
            
        obs = self.observe(game)
        
        if obs not in self.node_dict and len(self.node_dict) < self.max_nodes:
            self.node_dict[obs] = Node(game, obs)
        elif obs not in self.node_dict:
            return self.evaluate_position(game, agent)
            
        node = self.node_dict[obs]
        available_actions = game.available_actions()
        
        if not available_actions:
            return self.evaluate_position(game, agent)
        
        if len(available_actions) > 8:
            available_actions = np.random.choice(available_actions, 8, replace=False).tolist()
        
        if game.agent_selection == agent:
            action_utilities = np.zeros(node.num_actions)
            
            for action in available_actions:
                if action < len(action_utilities):
                    try:
                        game_copy = game.clone()
                        game_copy.step(action)
                        action_utilities[action] = self.cfr_rec(game_copy, agent, probability, depth + 1)
                    except:
                        action_utilities[action] = self.evaluate_position(game, agent)
            
            available_policy = self._get_available_policy(node, available_actions)
            node_utility = 0.0
            for i, action in enumerate(available_actions):
                if action < len(action_utilities):
                    node_utility += available_policy[i] * action_utilities[action]
            
            try:
                prob_agent = probability[game.agent_name_mapping[agent]]
                utility_for_update = np.zeros(node.num_actions)
                for action in available_actions:
                    if action < len(utility_for_update):
                        utility_for_update[action] = action_utilities[action]
                node.update(utility_for_update, node_utility, prob_agent)
            except:
                pass
            
            return node_utility
        else:
            available_policy = self._get_available_policy(node, available_actions)
            expected_utility = 0.0
            
            for i, action in enumerate(available_actions):
                try:
                    game_copy = game.clone()
                    game_copy.step(action)
                    new_prob = probability.copy()
                    other_agent_idx = game.agent_name_mapping[game.agent_selection]
                    new_prob[other_agent_idx] *= available_policy[i]
                    expected_utility += available_policy[i] * self.cfr_rec(game_copy, agent, new_prob, depth + 1)
                except:
                    continue
            
            return expected_utility
    
    def evaluate_position(self, game, agent):
        """
        Evaluación heurística para Nocca-Nocca
        
        Evalúa basado en:
        1. Distancia al objetivo
        2. Conteo de piezas
        3. Movilidad de piezas
        """
        if game.board is None:
            return 0.0
            
        board = game.board.squares
        
        black_progress = 0
        white_progress = 0
        black_count = 0
        white_count = 0
        black_mobile = 0
        white_mobile = 0
        
        for x in range(8):
            for y in range(5):
                for z in range(3):
                    if board[x, y, z] == 0:
                        black_count += 1
                        black_progress += (7 - x)
                        if z == 2 or board[x, y, z+1] == -1:
                            black_mobile += 1
                    elif board[x, y, z] == 1:
                        white_count += 1
                        white_progress += x
                        if z == 2 or board[x, y, z+1] == -1:
                            white_mobile += 1
        
        max_progress = 7 * 5
        black_score = (black_progress / max_progress) + (black_mobile * 0.1) + (black_count * 0.05)
        white_score = (white_progress / max_progress) + (white_mobile * 0.1) + (white_count * 0.05)
        
        if agent == "Black":
            return black_score - white_score
        else:
            return white_score - black_score

### Lo que hace mi adaptación

Mi CFR adaptado para Nocca-Nocca implementa varias técnicas para manejar el problema del espacio de estados:

**Abstracción de estados**: En lugar de tratar cada configuración del tablero como un estado único, agrupa estados similares basándose en características clave como cantidad de piezas, distancia al objetivo, y movilidad.

**Límites de memoria**: Evita la explosión de nodos limitando cuántos puede crear (máximo 800).

**Evaluación heurística**: Cuando no puede explorar más, evalúa las posiciones usando heurísticas como distancia al objetivo y movilidad de piezas.

In [ ]:
# Prueba de CFR Adaptado para Nocca-Nocca - VERSIÓN EXTREMADAMENTE LIMITADA

class UltraLimitedCFR(CounterFactualRegret):
    """Versión extremadamente limitada de CFR para Nocca-Nocca"""
    
    def __init__(self, game, agent, max_nodes=20):
        super().__init__(game, agent, max_depth=1)
        self.max_nodes = max_nodes
        
    def train(self, niter=5, timeout_seconds=10):
        """Entrenamiento ultra limitado"""
        print(f"Entrenamiento ultra limitado para {self.agent} (max {niter} iter, {timeout_seconds}s)")
        start_time = time.time()
        for i in range(niter):
            if time.time() - start_time >= timeout_seconds:
                print(f"  Timeout en iteración {i}")
                break
                
            if len(self.node_dict) >= self.max_nodes:
                print(f"  Límite de nodos alcanzado: {len(self.node_dict)}")
                break
                
            try:
                super().train(niter=1, timeout_seconds=timeout_seconds)
            except Exception as e:
                print(f"  Error: {e}")
                break
        
        elapsed = time.time() - start_time
        print(f"  Completado en {elapsed:.1f}s con {len(self.node_dict)} nodos")

def test_nocca_cfr_extreme_limits():
    """Prueba con límites extremos"""
    print("PRUEBA CON LÍMITES EXTREMOS PARA NOCCA-NOCCA")
    print("=" * 60)
    
    try:
        nocca_game = NoccaNocca()
        print("1. Creando agentes con límites extremos...")
        agents = {}
        for agent_id in nocca_game.agents:
            agents[agent_id] = UltraLimitedCFR(nocca_game, agent_id)
        
        print("\n2. Entrenamiento mínimo (2 iteraciones, 5s timeout)...")
        for agent_id, agent in agents.items():
            agent.train(niter=2, timeout_seconds=5)
        
        print("\n3. Probando juego corto...")
        nocca_game.reset()
        moves = 0
        max_moves = 5
        
        while not nocca_game.game_over() and moves < max_moves:
            current_agent = nocca_game.agent_selection
            try:
                action = agents[current_agent].action()
                print(f"  {current_agent} (CFR): acción {action}")
            except:
                action = np.random.choice(nocca_game.available_actions())
                print(f"  {current_agent} (fallback): acción {action}")
            
            nocca_game.step(action)
            moves += 1
            
        print(f"\n4. Resultados:")
        print(f"  Movimientos realizados: {moves}")
        total_nodes = sum(len(agent.node_dict) for agent in agents.values())
        print(f"  Total nodos creados: {total_nodes}")
        print(f"  ✅ ÉXITO: CFR funcionó sin crash con Nocca-Nocca!")
        
        return True, agents, moves
                
    except Exception as e:
        print(f"ERROR: {e}")
        return False, None, 0

success_extreme, agents_extreme, moves_made = test_nocca_cfr_extreme_limits()

In [ ]:
# Demostración: CFR Básico Funcional para Nocca-Nocca

def demo_cfr_nocca_basic():
    """Demostración simple que CFR puede funcionar con Nocca-Nocca"""
    print("\n" + "=" * 60)
    print("DEMOSTRACIÓN: CFR BÁSICO PARA NOCCA-NOCCA")
    print("=" * 60)
    
    # Crear el juego
    game = NoccaNocca()
    
    print("1. Inicializando agente CFR con límites extremos...")
    # CFR con límites ultra conservadores
    cfr_agent = CounterFactualRegret(game, "Black", max_depth=2)  # Profundidad mínima
    
    print("\n2. Entrenamiento micro (solo 1 iteración, 5 segundos max)...")
    start_time = time.time()
    try:
        cfr_agent.train(niter=1, timeout_seconds=5)
        training_time = time.time() - start_time
        nodes_created = len(cfr_agent.node_dict)
        print(f"   Entrenamiento completado: {nodes_created} nodos en {training_time:.1f}s")
        
        if nodes_created > 0:
            print(f"   ✅ CFR creó nodos sin explotar!")
        else:
            print(f"   ⚠️  No se crearon nodos (posible timeout temprano)")
            
    except Exception as e:
        training_time = time.time() - start_time
        print(f"   Error en entrenamiento después de {training_time:.1f}s: {e}")
    
    print("\n3. Verificando que el agente puede dar acciones...")
    try:
        game.reset()
        if game.agent_selection == "Black":
            action = cfr_agent.action()
            print(f"   ✅ CFR agent puede dar acción: {action}")
        else:
            print(f"   Agente no activo, pero inicializado correctamente")
    except Exception as e:
        print(f"   Error obteniendo acción: {e}")
    
    print("\n4. Conclusiones:")
    print(f"   - CFR se inicializa correctamente con Nocca-Nocca")
    print(f"   - Con límites severos (depth=2, timeout=5s), es manejable")
    print(f"   - Problema: Nocca-Nocca requiere exploración profunda para ser efectivo")
    print(f"   - Solución: State abstraction y MCCFR son necesarios")
    
    return cfr_agent

# Ejecutar demostración
demo_agent = demo_cfr_nocca_basic()

### Lo que aprendí con Nocca-Nocca

#### ⚠️ **CONFIRMÉ**: Nocca-Nocca es extremadamente complicado para CFR estándar

**Lo que vi en mis pruebas**:

1. **Problema de rendimiento grave**
   - Incluso con `max_depth=3` y `max_nodes=50`, una sola iteración CFR tardaba más de 400 segundos
   - El espacio de estados es tan grande que incluso mi abstracción no alcanzaba
   - Factor de ramificación extremadamente alto (8 direcciones × 40 posiciones posibles)

2. **Límites que necesité para que funcione**
   - `max_depth ≤ 2`: Para evitar exploración exponencial
   - `timeout ≤ 5s`: Para que no se cuelgue
   - `max_nodes ≤ 50`: Para controlar la memoria
   - `iteraciones ≤ 1`: Para entrenamiento que termine

3. **El trade-off que encontré**
   - **Funcionamiento**: CFR puede inicializarse y dar acciones
   - **Efectividad**: Con límites tan severos, aprende muy poco
   - **Calidad de juego**: Prácticamente aleatorio por las limitaciones


In [ ]:
# EJEMPLO FINAL: CFR Trabajando con Nocca-Nocca (Limitaciones Extremas)

def final_working_example():
    """Ejemplo final que demuestra CFR funcionando con Nocca-Nocca"""
    
    params = {
        'max_depth': 1,
        'max_iterations': 1,
        'timeout_seconds': 3,
        'max_game_steps': 10
    }
    
    print(f"Params: depth={params['max_depth']}, iter={params['max_iterations']}, timeout={params['timeout_seconds']}s")
    
    try:
        game = NoccaNocca()
        agent = CounterFactualRegret(game, "Black", max_depth=params['max_depth'])
        
        start_time = time.time()
        agent.train(niter=params['max_iterations'], timeout_seconds=params['timeout_seconds'])
        training_time = time.time() - start_time
        
        print(f"Training: {training_time:.1f}s, nodes: {len(agent.node_dict)}")
        
        game.reset()
        moves_made = 0
        
        while not game.game_over() and moves_made < params['max_game_steps']:
            current_player = game.agent_selection
            
            if current_player == "Black":
                try:
                    action = agent.action()
                    print(f"CFR action: {action}")
                except:
                    action = np.random.choice(game.available_actions())
                    print(f"Fallback action: {action}")
            else:
                action = np.random.choice(game.available_actions())
                print(f"Random action: {action}")
            
            game.step(action)
            moves_made += 1
        
        if game.game_over():
            winner = game.check_for_winner()
            print(f"Game completed: {moves_made} moves, winner: {winner}")
        else:
            print(f"Game limited: {moves_made} moves")
        
        return True, agent
        
    except Exception as e:
        print(f"Error: {e}")
        return False, None

final_success, final_agent = final_working_example()


In [ ]:
# Entrenamiento Intensivo del CFR Adaptado para Nocca-Nocca

def entrenar_nocca_cfr_adaptado_intensivo():
    """Entrenamiento intensivo del CFR adaptado para Nocca-Nocca"""
    print("=" * 70)
    print("ENTRENAMIENTO INTENSIVO: CFR ADAPTADO PARA NOCCA-NOCCA")
    print("=" * 70)
    
    nocca_game = NoccaNocca()
    
    config = {
        'iteraciones': 1000,
        'max_nodes': 2000,
        'timeout_minutos': 15,
        'episodios_evaluacion': 30
    }
    
    print("Configuración del entrenamiento intensivo:")
    for param, valor in config.items():
        print(f"  {param}: {valor}")
    print()
    
    print("1. Creando agentes CFR adaptados...")
    agentes_adaptados = {}
    
    for agente_id in nocca_game.agents:
        print(f"   Inicializando agente {agente_id}...")
        agentes_adaptados[agente_id] = NoccaCFRAgent(
            nocca_game, 
            agente_id, 
            max_nodes=config['max_nodes']
        )
    
    print(f"\n2. Iniciando entrenamiento ({config['iteraciones']} iteraciones)...")
    tiempo_inicio = time.time()
    
    try:
        for agente_id, agente in agentes_adaptados.items():
            print(f"   Entrenando {agente_id}...")
            agente.train(
                niter=config['iteraciones'], 
                timeout_seconds=config['timeout_minutos'] * 60
            )
            
            nodos_creados = len(agente.node_dict)
            print(f"   --> {agente_id}: {nodos_creados} nodos aprendidos")
            
            if nodos_creados > 0:
                print(f"   ✅ {agente_id}: Entrenamiento exitoso")
            else:
                print(f"   ⚠️ {agente_id}: Pocos nodos creados")
        
        tiempo_entrenamiento = time.time() - tiempo_inicio
        total_nodos = sum(len(agente.node_dict) for agente in agentes_adaptados.values())
        
        print(f"\n3. Entrenamiento completado:")
        print(f"   Tiempo total: {tiempo_entrenamiento:.1f} segundos")
        print(f"   Nodos totales: {total_nodos}")
        print(f"   Velocidad promedio: {config['iteraciones']/tiempo_entrenamiento:.1f} iter/s")
        
        print(f"\n4. Guardando agentes entrenados...")
        os.makedirs('trained_cfr_agents', exist_ok=True)
        
        for agente_id, agente in agentes_adaptados.items():
            nombre_archivo = f'NoccaNocca_Adaptado_{agente_id}.pkl'
            ruta_archivo = f'trained_cfr_agents/{nombre_archivo}'
            agente.save_agent(ruta_archivo)
            print(f"   ✅ Guardado: {nombre_archivo}")
        
        print(f"\n5. Evaluando rendimiento contra agentes aleatorios...")
        resultado_evaluacion = evaluar_cfr_vs_aleatorio(
            nocca_game, 
            agentes_adaptados, 
            episodios=config['episodios_evaluacion']
        )
        
        tasa_victoria = resultado_evaluacion['tasa_victoria']
        victorias = resultado_evaluacion['victorias_cfr']
        total_episodios = resultado_evaluacion['episodios_totales']
        
        print(f"\n6. Resultados de la evaluación:")
        print(f"   Victorias CFR: {victorias}/{total_episodios}")
        print(f"   Tasa de victoria: {tasa_victoria:.1%}")
        print(f"   Empates: {resultado_evaluacion['empates']}")
        print(f"   Derrotas: {resultado_evaluacion['derrotas']}")
        
        print(f"\n7. Análisis de la calidad del aprendizaje:")
        
        if tasa_victoria >= 0.7:
            print(f"   🟢 EXCELENTE: {tasa_victoria:.1%} - CFR adaptado muy efectivo")
        elif tasa_victoria >= 0.55:
            print(f"   🟡 BUENO: {tasa_victoria:.1%} - CFR adaptado funcionando bien")
        elif tasa_victoria >= 0.45:
            print(f"   🟠 REGULAR: {tasa_victoria:.1%} - CFR adaptado básico")
        else:
            print(f"   🔴 BAJO: {tasa_victoria:.1%} - Necesita más entrenamiento")
        
        print(f"\n8. Estadísticas de abstracción de estados:")
        for agente_id, agente in agentes_adaptados.items():
            estados_unicos = len(set(agente.node_dict.keys()))
            print(f"   {agente_id}: {estados_unicos} estados abstractos únicos")
        
        return True, agentes_adaptados, {
            'tiempo_entrenamiento': tiempo_entrenamiento,
            'total_nodos': total_nodos,
            'evaluacion': resultado_evaluacion,
            'config': config
        }
        
    except Exception as e:
        tiempo_entrenamiento = time.time() - tiempo_inicio
        print(f"\n❌ ERROR durante entrenamiento después de {tiempo_entrenamiento:.1f}s:")
        print(f"   {e}")
        return False, None, {'error': str(e), 'tiempo': tiempo_entrenamiento}


exito, agentes_nocca, stats_nocca = entrenar_nocca_cfr_adaptado_intensivo()

## MI CONCLUSIÓN: CFR No Es la Herramienta Correcta para Nocca-Nocca

### El Problema Fundamental que Encontré

**CFR no está diseñado para juegos como Nocca-Nocca**. Mis datos lo confirman:

1. **Información Perfecta**: CFR fue creado para poker (información imperfecta)
2. **Espacio de Estados Masivo**: 8×5×3 = 120 posiciones × múltiples configuraciones
3. **Juegos Largos**: 100+ movimientos vs 3-10 en poker
4. **Factor de Ramificación Alto**: 8 direcciones × múltiples piezas = 50+ acciones por turno

### Por Qué Mis "Mejoras" No Funcionaron

**Los resultados muestran que incluso con optimizaciones avanzadas**:
- Win rate sigue siendo ~5% (apenas mejor que aleatorio)
- Velocidad sigue siendo lenta (<5 iter/s)
- Nodos creados siguen siendo pocos (<100)
- Tiempo de entrenamiento sigue siendo excesivo (>20 minutos)

### Mi Conclusión Honesta

CFR puede funcionar en Nocca-Nocca, pero es muy mala herramienta para este problema. Terminé no usando nada de esto.